The following code defines LSTM model setup trained on visual modality data. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

In [ ]:
import sys
sys.path.insert(1, '../')

from data_preparation import prepare_x_data, get_Y_labels, reshape_Y, reshape_X, unscale_Y

In [ ]:
scaler = StandardScaler()

## Data preparation

Data preparation includes: <br>
<ul>
  <li>selecting necessary features from source files</li>
  <li>creating combined dataset for the model training</li>
  <li>reshaping data for model training.</li>


### X data - visual features

In [ ]:
x_train = prepare_x_data('../Data/LLDs_video_openface/train',',', 5, scaler)

In [ ]:
x_test = prepare_x_data('../Data/LLDs_video_openface/dev',',', 5, scaler) 

### Y data - YMRS score

In [ ]:
y_train = get_Y_labels('../Data/labels_metadata.csv', 60, 164, scaler)
y_train = reshape_Y(y_train,len(x_train),1,1)

In [ ]:
y_test = get_Y_labels('../Data/labels_metadata.csv', 0, 60, scaler)
y_test = reshape_Y(y_test,len(x_test),1,1)

## Model setup - LSTM

Proposed model for LSTM recurrent neural network architecture.

### Defining the model

In [ ]:
model = keras.Sequential()

In [ ]:
model.add(layers.LSTM(units = 206, input_shape=(None,465), return_sequences=True))
model.add(layers.Dropout(0.004))
model.add(layers.LSTM(units = 206, input_shape=(None,206), return_sequences=False))
model.add(layers.Dropout(0.004))
model.add(layers.Dense(1, activation='linear'))

In [ ]:
model.compile(
    loss="mse", 
    metrics= [keras.metrics.MeanAbsoluteError()], #['mean_absolute_error'],  
    optimizer=keras.optimizers.Adam(learning_rate=0.003)
    )

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model=model, show_dtype=True, show_layer_names=True, show_shapes=True, to_file='LSTM_unimodal_visual.png') # learning rate

### Model training

Model training setup is based on an iterative approach where model is trained one file at a time, then learned parameters are saved and loaded in the next iterative step. This setup is necessary due to the fact that source files does not have an uniform size and differ in number of frames.

In [ ]:
train_index = 0

train_eval = []
train_loss = []
train_mae = []

for file in x_train:
        file = np.array(file).reshape((1, file.shape[0], -1))
        model.fit(x=file, 
                y=y_train[train_index], 
                epochs=10,
                batch_size = len(file))    
                
        scores = model.evaluate(file, y_train[train_index], verbose = 0)
        train_eval.append(scores)
        train_loss.append(scores[0])
        train_mae.append(scores[1])
                
        model.save(r'LSTM_train_visual', include_optimizer=True) # Save model configuration to Saved_models.
        model = keras.models.load_model(r'LSTM_train_visual') # Load model configuration from Saved_models.
                
        train_index += 1  

#### Model evaluation - train set

In [ ]:
avg_train_loss = sum(train_loss) / len(train_loss)
avg_train_mae = sum(train_mae) / len(train_mae)
print("Train loss (avg):", avg_train_loss, "Train MAE (avg):", avg_train_mae)

In [ ]:
real_train_loss = round(float(scaler.inverse_transform(np.array(avg_train_loss).reshape(-1,1))), 3)
real_train_mae = round(float(scaler.inverse_transform(np.array(avg_train_mae).reshape(-1,1))), 3)

print(f'AVG MSE: {real_train_loss}')
print(f'AVG MAE: {real_train_mae}')

In [ ]:
plt.xlabel("Iterations")
plt.ylabel("Scaled YMRS value")
plt.plot(train_loss, label="MSE")
plt.plot(train_mae, label="MAE")
plt.legend()
plt.show()

### Model evaluation  - validation set

Subset taken from test data is defined as a validation set.

In [ ]:
x_val, y_val = x_test[:30], y_test[:30]

In [ ]:
eval_loss = []
eval_mae = []
eval_index = 0

for input in x_val:
    input = reshape_X(input)
    scores = model.evaluate(input, y_val[eval_index], verbose = 0)
    
    eval_loss.append(scores[0])
    eval_mae.append(scores[1])
    
    eval_index += 1

In [ ]:
avg_eval_loss = sum(eval_loss) / len(eval_loss)
avg_eval_mae = sum(eval_mae) / len(eval_mae)
print("Validation loss (avg):", avg_eval_loss, "Validation MAE (avg):", avg_eval_mae)

In [ ]:
real_eval_loss = round(float(scaler.inverse_transform(np.array(avg_eval_loss).reshape(-1,1))), 3)
real_eval_mae = round(float(scaler.inverse_transform(np.array(avg_eval_mae).reshape(-1,1))), 3)

print(f'AVG MSE: {real_eval_loss}')
print(f'AVG MAE: {real_eval_mae}')

In [ ]:
plt.xlabel("Iterations")
plt.ylabel("Scaled YMRS value")
plt.plot(eval_loss, label="MSE")
plt.plot(eval_mae, label="MAE")
plt.legend()
plt.show()

## Prediction

Prediction is made on different subset taken from the test dataset. Then actual and predicted YMRS values are compared.

In [ ]:
x_pred, y_pred_actual = x_test[30:], y_test[30:]

In [ ]:
y_prediction = []

pred_scores = []
pred_loss = []
pred_mae = []

pred_index = 0

for file in x_pred:
    file = np.array(file).reshape(1, file.shape[0], -1)
    pred_y = y_prediction.append(model.predict(file))
    
    scores = model.evaluate(file, y_pred_actual[pred_index], verbose=0)
    pred_scores.append(scores)
    pred_loss.append(scores[0])
    pred_mae.append(scores[1])
    
    pred_index += 1

### Model evaluation - prediction set

In [ ]:
avg_pred_loss = sum(pred_loss) / len(pred_loss)
avg_pred_mae = sum(pred_mae) / len(pred_mae)
print("Prediction loss (avg):", avg_pred_loss, "Prediction MAE (avg):", avg_pred_mae)

In [ ]:
real_pred_loss = round(float(scaler.inverse_transform(np.array(avg_pred_loss).reshape(-1,1))), 3)
real_pred_mae = round(float(scaler.inverse_transform(np.array(avg_pred_mae).reshape(-1,1))), 3)

print(f'AVG MSE: {real_pred_loss}')
print(f'AVG MAE: {real_pred_mae}')

In [ ]:
plt.xlabel("Iterations")
plt.ylabel("Scaled YMRS value")
plt.plot(pred_loss, label="MSE")
plt.plot(pred_mae, label="MAE")
plt.legend()
plt.show()

### Actual vs predicted comparison

In [ ]:
y_prediction = np.array(y_prediction).reshape(-1, 1)
y_pred_actual = np.array(y_pred_actual).reshape(-1,1)

y_prediction = unscale_Y(y_prediction, scaler)
y_pred_actual = unscale_Y(y_pred_actual, scaler)

In [ ]:
pred_df = pd.DataFrame(data=np.column_stack((y_pred_actual,y_prediction)),columns=['y_actual','y_pred'])
pred_df['pred_error'] = pred_df['y_actual'] - pred_df['y_pred']
pred_df = pred_df.sort_values(by=['y_actual']).reset_index()
pred_df['y_actual'] = pred_df['y_actual'].apply(np.int64)
pred_df

In [ ]:
pred_df.plot('y_actual', 'y_pred', kind='scatter')
plt.xlabel("Actual YMRS")
plt.ylabel("Predicted YMRS")
plt.show()

In [ ]:
pred_df.plot('y_actual', 'pred_error', kind='scatter')
plt.xlabel("Actual YMRS")
plt.ylabel("Prediction error")
plt.show()

In [ ]:
plt.xlabel("Iterations")
plt.ylabel("Target value (YMRS)")
plt.plot(pred_df['y_actual'], label="Actual")
plt.plot(pred_df['y_pred'], label="Predicted")
plt.legend()
plt.show()